In [1]:
# FINAL PROJECT DATA ENGINEERING (Short Trade Program Script)
# Author - Priya Kundu and Nikitaa Kenkre
# Date of Program Run - 11th May 2023 (3pm to 7:30am)
# Date of Presentation - 12th May 2023 (1pm)
# Semester Spring 2023
# By Professor Carlos De Oliveira

# Importing Packages and Setting Parameters

In [2]:
import oandapyV20
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import pandas as pd
import datetime as dt
import time
from datetime import datetime

In [3]:
# OANDA account details
access_token = "9a1d828e9ba37770b2cba4e452889987-ab8fd896349ffa39f6b962eacafb7ac9"
accountID = "101-001-25499777-002"
#api = oandapyV20.API(access_token=access_token, environment="practice")
try:
    client = oandapyV20.API(access_token=access_token, environment="practice")
except V20Error as e:
    print("Error: {}".format(e))

In [4]:
# specify the instrument to trade
instrument = "USD_CAD" 

In [5]:
# set the order parameters
order_quantity = 1000 # start with 10k units
total_order_quantity = 100000 # the total order quantity
current_executed_quantity = 0 # keep track of how many units have been executed so far
non_executed_quantity = 0 # keep track of the units that were not executed in the previous window
average_execution_price = 0 # initialize the average execution price to 0
total_average_execution_price = 0 # initialize the total average execution price to 0

In [6]:
# setting the required time frames
first_execution_start_time = dt.datetime.now().replace(hour=15, minute=0, second=0, microsecond=0)
first_execution_end_time = dt.datetime.now().replace(hour=17, minute=0, second=0, microsecond=0)

second_execution_start_time = dt.datetime.now().replace(hour=19, minute=0, second=0, microsecond=0)
second_execution_end_time = dt.datetime.now().replace(hour=22, minute=0, second=0, microsecond=0)

third_execution_start_time = dt.datetime.now().replace(hour=23, minute=0, second=0, microsecond=0)
third_execution_end_time = dt.datetime.now().replace(hour=1, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time

fourth_execution_start_time = dt.datetime.now().replace(hour=3, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the start time
fourth_execution_end_time = dt.datetime.now().replace(hour=6, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time

In [7]:
# define a function to execute orders
def execute_order(units, instrument, side, trades):
    # create a market order request
    direction = 1 if side == "BUY" else -1
    mo = MarketOrderRequest(
        instrument=instrument,
        units=str(int(units)*direction),
        takeProfitOnFill=None,
        stopLossOnFill=None,
    )
    # send the order request
    r = orders.OrderCreate(accountID, data=mo.data)
    response = client.request(r)
    if "orderFillTransaction" in response:
        trades.append({
            "Timestamp": dt.datetime.now(), 
            "Order_ID": float(response["orderFillTransaction"]["orderID"]),
            "Instrument": instrument,
            "Price": float(response["orderFillTransaction"]["price"]),
            "Units": int(units),
            "Side": side
        })
    else:
        # Handle the case where the order is not filled
        if "orderCancelTransaction" in response:
            print("Order canceled due to:", response["orderCancelTransaction"]["reason"])
        else:
            print("No cancel transaction found.")

In [8]:
# make a function to get price that can be used to compare with the average price before execution takes place
def get_rate():
    price_request = pricing.PricingInfo(accountID=accountID, params={'instruments': instrument})
    response = client.request(price_request)
    return float(response['prices'][0]['asks'][0]['price'])

# First Interval

In [9]:
# execute the first window
while dt.datetime.now() < first_execution_start_time:
    time.sleep(1)

batch1_trades = []  # create a list to store the prices
while dt.datetime.now() >= first_execution_start_time and dt.datetime.now() <= first_execution_end_time:
    for i in range(20): # 20 iterations to execute 10k units every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.2:       
            response = execute_order(order_quantity, instrument, "SELL", batch1_trades)
            current_executed_quantity += order_quantity
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval") 

Successful execution for 0 th interval
Successful execution for 1 th interval
Successful execution for 2 th interval
Successful execution for 3 th interval
Successful execution for 4 th interval
Successful execution for 5 th interval
Successful execution for 6 th interval
Successful execution for 7 th interval
Successful execution for 8 th interval
Successful execution for 9 th interval
Successful execution for 10 th interval
Successful execution for 11 th interval
Successful execution for 12 th interval
Successful execution for 13 th interval
Successful execution for 14 th interval
Successful execution for 15 th interval
Successful execution for 16 th interval
Successful execution for 17 th interval
Successful execution for 18 th interval
Successful execution for 19 th interval


In [10]:
# convert the list of prices into a DataFrame
df1 = pd.DataFrame(batch1_trades)

In [11]:
df1

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 15:00:00.604268,649.0,USD_CAD,1.34861,1000,SELL
1,2023-05-11 15:06:00.733932,655.0,USD_CAD,1.34899,1000,SELL
2,2023-05-11 15:12:00.819525,661.0,USD_CAD,1.34882,1000,SELL
3,2023-05-11 15:18:00.910598,667.0,USD_CAD,1.34878,1000,SELL
4,2023-05-11 15:24:01.021582,673.0,USD_CAD,1.34884,1000,SELL
5,2023-05-11 15:30:01.110899,679.0,USD_CAD,1.34882,1000,SELL
6,2023-05-11 15:36:01.230583,685.0,USD_CAD,1.34863,1000,SELL
7,2023-05-11 15:42:01.338860,691.0,USD_CAD,1.34892,1000,SELL
8,2023-05-11 15:48:01.431862,697.0,USD_CAD,1.34891,1000,SELL
9,2023-05-11 15:54:01.577189,703.0,USD_CAD,1.34907,1000,SELL


In [12]:
average_execution_price1 = df1['Price'].sum() / len(df1)

In [13]:
average_execution_price1

1.348932

In [14]:
current_executed_quantity

20000

# Second Interval

In [15]:
# execute the second window
while dt.datetime.now() < second_execution_start_time:
    time.sleep(1)

batch2_trades = []  # create a list to store the prices

while dt.datetime.now() >= second_execution_start_time and dt.datetime.now() <= second_execution_end_time:
    for i in range(30): # 30 iterations to execute 10k units every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.5:
            if get_rate() <= average_execution_price1:
                execute_order(order_quantity, instrument, "SELL", batch2_trades)
                current_executed_quantity += order_quantity
                print("Trade took place!")
            else:
                print("Trade did not take place as Price < Average Price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

Trade did not take place as Price < Average Price
Successful execution for 0 th interval
Trade did not take place as Price < Average Price
Successful execution for 1 th interval
Trade did not take place as Price < Average Price
Successful execution for 2 th interval
Trade did not take place as Price < Average Price
Successful execution for 3 th interval
Trade did not take place as Price < Average Price
Successful execution for 4 th interval
Trade did not take place as Price < Average Price
Successful execution for 5 th interval
Trade did not take place as Price < Average Price
Successful execution for 6 th interval
Trade did not take place as Price < Average Price
Successful execution for 7 th interval
Trade did not take place as Price < Average Price
Successful execution for 8 th interval
Trade did not take place as Price < Average Price
Successful execution for 9 th interval
Trade did not take place as Price < Average Price
Successful execution for 10 th interval
Trade did not take p

In [16]:
# convert the list of prices into a DataFrame
df2 = pd.DataFrame(batch2_trades)

In [17]:
df2

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 21:00:03.118306,770.0,USD_CAD,1.34856,1000,SELL
1,2023-05-11 21:06:03.270214,773.0,USD_CAD,1.34834,1000,SELL
2,2023-05-11 21:12:03.412901,776.0,USD_CAD,1.34857,1000,SELL
3,2023-05-11 21:18:03.574961,779.0,USD_CAD,1.34826,1000,SELL
4,2023-05-11 21:24:03.648774,782.0,USD_CAD,1.34834,1000,SELL
5,2023-05-11 21:30:03.809569,785.0,USD_CAD,1.34856,1000,SELL
6,2023-05-11 21:36:03.976010,788.0,USD_CAD,1.34847,1000,SELL
7,2023-05-11 21:42:04.148836,791.0,USD_CAD,1.34860,1000,SELL
8,2023-05-11 21:48:04.342321,794.0,USD_CAD,1.34858,1000,SELL
9,2023-05-11 21:54:04.548703,797.0,USD_CAD,1.34850,1000,SELL


In [18]:
# concatenate the two dataframes vertically
new_df = pd.concat([df1, df2], axis=0)

In [19]:
# calculate the total average execution price
average_execution_price2 = new_df['Price'].sum() / len(new_df)

average_execution_price2

1.3487806666666664

In [20]:
current_executed_quantity

30000

In [21]:
non_executed_quantity2 = total_order_quantity * 0.5 - current_executed_quantity

non_executed_quantity2

20000.0

# Third Interval

In [23]:
# execute the third window
while dt.datetime.now() < third_execution_start_time:
    time.sleep(1)

batch3_trades = []  # create a list to store the prices

while dt.datetime.now() >= third_execution_start_time and dt.datetime.now() <= third_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity2 + (total_order_quantity * 0.2)) / 20)
    for i in range(20): # 20 iterations to execute every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.7:
            if get_rate() <= average_execution_price2:
                response = execute_order(order_quantity, instrument, "SELL", batch3_trades)
                current_executed_quantity += order_quantity
                print("Trade took place!")
            else:
                print("Trade did not take place as Price < Average Price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

Trade did not take place as Price < Average Price
Successful execution for 0 th interval
Trade did not take place as Price < Average Price
Successful execution for 1 th interval
Trade did not take place as Price < Average Price
Successful execution for 2 th interval
Trade did not take place as Price < Average Price
Successful execution for 3 th interval
Trade did not take place as Price < Average Price
Successful execution for 4 th interval
Trade did not take place as Price < Average Price
Successful execution for 5 th interval
Trade did not take place as Price < Average Price
Successful execution for 6 th interval
Trade did not take place as Price < Average Price
Successful execution for 7 th interval
Trade did not take place as Price < Average Price
Successful execution for 8 th interval
Trade did not take place as Price < Average Price
Successful execution for 9 th interval
Trade did not take place as Price < Average Price
Successful execution for 10 th interval
Trade did not take p

In [24]:
# convert the list of prices into a DataFrame
df3 = pd.DataFrame(batch3_trades)

In [25]:
df3

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-12 00:48:03.465455,800.0,USD_CAD,1.34849,2000,SELL


In [26]:
# concatenate the two dataframes vertically
updated_df = pd.concat([new_df, df3], axis=0)

In [27]:
# calculate the total average execution price
average_execution_price3 = updated_df['Price'].sum() / len(updated_df)

average_execution_price3

1.3487712903225804

In [28]:
current_executed_quantity

32000

In [29]:
non_executed_quantity3 = total_order_quantity * 0.7 - current_executed_quantity

non_executed_quantity3

38000.0

# Fourth Interval

In [30]:
# execute the fourth window
while dt.datetime.now() < fourth_execution_start_time:
    time.sleep(1)

batch4_trades = []  # create a list to store the prices

while dt.datetime.now() >= fourth_execution_start_time and dt.datetime.now() <= fourth_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity3 + (total_order_quantity * 0.3)) / 30)
    for i in range(30): # 30 iterations to execute every 6 minutes
        if current_executed_quantity < total_order_quantity:
            if get_rate() <= average_execution_price3:
                response = execute_order(order_quantity, instrument, "SELL", batch4_trades)
                current_executed_quantity += order_quantity
                print("Trade took place!")
            else:
                print("Trade did not take place as Price > Average Price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

Trade did not take place as Price > Average Price
Successful execution for 0 th interval
Trade did not take place as Price > Average Price
Successful execution for 1 th interval
Trade did not take place as Price > Average Price
Successful execution for 2 th interval
Trade did not take place as Price > Average Price
Successful execution for 3 th interval
Trade took place!
Successful execution for 4 th interval
Trade took place!
Successful execution for 5 th interval
Trade did not take place as Price > Average Price
Successful execution for 6 th interval
Trade took place!
Successful execution for 7 th interval
Trade took place!
Successful execution for 8 th interval
Trade did not take place as Price > Average Price
Successful execution for 9 th interval
Trade did not take place as Price > Average Price
Successful execution for 10 th interval
Trade did not take place as Price > Average Price
Successful execution for 11 th interval
Trade did not take place as Price > Average Price
Successf

In [31]:
# convert the list of prices into a DataFrame
df4 = pd.DataFrame(batch4_trades)

In [32]:
df4

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-12 03:24:02.068240,803.0,USD_CAD,1.34826,2266,SELL
1,2023-05-12 03:30:02.243895,806.0,USD_CAD,1.34833,2266,SELL
2,2023-05-12 03:42:02.570119,809.0,USD_CAD,1.34815,2266,SELL
3,2023-05-12 03:48:02.723507,812.0,USD_CAD,1.34846,2266,SELL
4,2023-05-12 04:30:03.895234,815.0,USD_CAD,1.34857,2266,SELL
5,2023-05-12 05:06:04.879661,818.0,USD_CAD,1.34850,2266,SELL
6,2023-05-12 05:12:05.030121,821.0,USD_CAD,1.34850,2266,SELL
7,2023-05-12 05:30:05.353611,824.0,USD_CAD,1.34853,2266,SELL
8,2023-05-12 05:36:05.530955,827.0,USD_CAD,1.34856,2266,SELL


In [33]:
# concatenate the two dataframes vertically
final_df = pd.concat([updated_df, df4], axis=0)

In [34]:
# calculate the total average execution price
final_average_execution_price = updated_df['Price'].sum() / len(updated_df)

final_average_execution_price

1.3487712903225804

In [35]:
current_executed_quantity

52394

In [36]:
non_executed_quantity_final = total_order_quantity - current_executed_quantity

non_executed_quantity_final

47606

In [37]:
final_df.to_csv("Long_Final_Executed_Data.csv", index=False)

# Final Extensions

In [38]:
final_df

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 15:00:00.604268,649.0,USD_CAD,1.34861,1000,SELL
1,2023-05-11 15:06:00.733932,655.0,USD_CAD,1.34899,1000,SELL
2,2023-05-11 15:12:00.819525,661.0,USD_CAD,1.34882,1000,SELL
3,2023-05-11 15:18:00.910598,667.0,USD_CAD,1.34878,1000,SELL
4,2023-05-11 15:24:01.021582,673.0,USD_CAD,1.34884,1000,SELL
5,2023-05-11 15:30:01.110899,679.0,USD_CAD,1.34882,1000,SELL
6,2023-05-11 15:36:01.230583,685.0,USD_CAD,1.34863,1000,SELL
7,2023-05-11 15:42:01.338860,691.0,USD_CAD,1.34892,1000,SELL
8,2023-05-11 15:48:01.431862,697.0,USD_CAD,1.34891,1000,SELL
9,2023-05-11 15:54:01.577189,703.0,USD_CAD,1.34907,1000,SELL


In [39]:
current_executed_quantity

52394

In [40]:
non_executed_quantity_final

47606

In [41]:
if non_executed_quantity_final > 0:
    # Wait for 30 minutes after last execution window
    extension1_end_time = fourth_execution_end_time + dt.timedelta(minutes=30)
    while dt.datetime.now() < extension1_end_time:
        time.sleep(60)

    extension1_executed_trades = []
    new_order_quantity = int(non_executed_quantity_final * 0.5)

    # Check condition and execute trade
    if get_rate() <= final_average_execution_price:
        execute_order(new_order_quantity, instrument, "SELL", extension1_executed_trades)
        print("Traded 50% of the Non Executed Quantity in one go as Price <= Average Price")
        
    else:
        print("Trading did not take place as Price > Average Price.")
        
else:
    print("Trading did not take place as the entire quantity was executed in the previous four execution windows.")

Trading did not take place as Price > Average Price.


In [42]:
# convert the list of prices into a DataFrame
extension1_df = pd.DataFrame(extension1_executed_trades)

extension1_df

""


In [43]:
if non_executed_quantity_final > 0:
    
    # Wait for 60 minutes after last execution window
    extension2_end_time = fourth_execution_end_time + dt.timedelta(minutes=60)
    while dt.datetime.now() < extension2_end_time:
        time.sleep(60)

    extension2_executed_trades = []

    # Check condition and execute trade
    if not extension1_df.empty:
        # If current Price is more than the Average Execution Price
        if get_rate() <= final_average_execution_price:
            # Buy remaining 50% of non-executed amount
            execute_order(new_order_quantity, instrument, "SELL", extension2_executed_trades)
            print("Traded remaining 50% of the Non Executed Quantity in one go as Price <= Average Price and Extention1 was executed.")
        # If If current Price is less than the Average Execution Price
        else:
            # Buy remaining 50% of non-executed amount
            execute_order(new_order_quantity, instrument, "SELL", extension2_executed_trades)
            print("Traded remaining 50% of the Non Executed Quantity in one go as Price > Average Price and Extention1 was executed.")
     
    else:
        # Check if current price is below average price, then start selling
        if get_rate() > final_average_execution_price:
            # Sell total executed units every one-minute (TWAP) whatever the price is until 20% of initial order is reached
            required_quantity = current_executed_quantity - (total_order_quantity * 0.2)
            executed_quantity = 0

            # Keep selling until executed_quantity < new_order_size
            while executed_quantity < required_quantity:
                execute_order(order_quantity, instrument, "BUY", extension2_executed_trades)
                executed_quantity += order_quantity
                time.sleep(60)
                print("Successful execution!")
            print("Order quantity reached below 20% of the initial order.")
            
        else:
            print ("Trading did not take place as Price < Average Price.")
            
else:
    print("Trading did not take place as the entire quantity was executed in the previous four execution windows.")

Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Successful execution!
Order quantity reached below 20% of the initial order.


In [44]:
# convert the list of prices into a DataFrame
extension2_df = pd.DataFrame(extension2_executed_trades)

extension2_df

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-12 07:00:33.096415,830.0,USD_CAD,1.34941,2266,BUY
1,2023-05-12 07:01:33.197617,832.0,USD_CAD,1.34946,2266,BUY
2,2023-05-12 07:02:33.274513,834.0,USD_CAD,1.34945,2266,BUY
3,2023-05-12 07:03:33.392220,836.0,USD_CAD,1.34947,2266,BUY
4,2023-05-12 07:04:33.836028,838.0,USD_CAD,1.34956,2266,BUY
5,2023-05-12 07:05:33.951338,840.0,USD_CAD,1.34954,2266,BUY
6,2023-05-12 07:06:34.080624,842.0,USD_CAD,1.34955,2266,BUY
7,2023-05-12 07:07:34.228855,844.0,USD_CAD,1.34949,2266,BUY
8,2023-05-12 07:08:34.350781,846.0,USD_CAD,1.34954,2266,BUY
9,2023-05-12 07:09:34.456223,848.0,USD_CAD,1.34935,2266,BUY


# Updating the Database

In [45]:
from pymongo import MongoClient

In [46]:
# MongoDb Setup
client = MongoClient('mongodb://localhost:27017/')
db = client["TWAP_Forex_Trading"]

In [47]:
collection_name = "Short_Trade_USDCAD" 
collection = db[collection_name]
collection.insert_many(final_df.to_dict("records") + extension1_df.to_dict("records") + extension2_df.to_dict("records"))

# Creating required csv File

In [51]:
# Create a dictionary with your data
data = {'Interval': ['Interval 1', 'Interval 2', 'Interval 3', 'Interval 4'],
        'Executed Quantity': [200000, 10000, 2000, 20394], 
        'Percentage of Executed Quantity': [100, 33.33, 10, 67.98],
        'Non Executed Quantity': [0, 20000, 18000, 9606],
        'Percentage of Non Executed Quantity': [0, 66.67, 90, 32.02]}

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(data)

# Add some sample data for the last three columns
df['Average Execution Price'] = [average_execution_price1, average_execution_price2, average_execution_price3, final_average_execution_price]

# Convert the DataFrame to a CSV file
df.to_csv('Final Project - Short - Execution Output Vectors.csv', index=False)